In [2]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

model = "mistral-large-latest"

api_key = 'WbwKVXrrMHUqV5BWUzUB5YBWjOGEUWWw'
client = MistralClient(api_key=api_key)

def chat_mistral(prompt: str):
   messages = [
       ChatMessage(role="user", content=prompt)
   ]

   # No streaming
   chat_response = client.chat(
       model=model,
       messages=messages,
   )

   return chat_response.choices[0].message.content

In [55]:
import json

def get_mistral_response(argobot_answer, question, context):
    prompt = '''return a single number in json format. Do not return anything except json'''
    prompt = f'''
        # Introduction
        You are a specialist RAG evaluation.
        Your task is to evaluate another chatbot's answer.
        
        # Instruction
        After receiving the another chatbot's answer, you must return 0 if the answer is not in the context, and 1 vice versa.
        Keep in mind. Argobot answer and context is in Vietnamese, so you should translate it to English before evaluation.
        You can also return 1 if:
            - If the question is irrelevant to the context.
            - If the chatbot answer is no because of the lack of knowledge, and the context actually does not contain the answer for the question.
        Don't be too harsh. A question is relevant if it contains the some keywords in the context.
        
        Answer is: <argobot_answer>{argobot_answer} <argobot_answer>
        Question is: <question>{question} <question>
        Context is: <context>{context}<context>
        
        You must respond in the following structured JSON format, all fields are mandatory:
        {{
            "score": 0 or 1 based on the relevance of answer to the context
        }}
    '''
    score_json = chat_mistral(prompt)
    score = json.loads(score_json)['score']
    return score
# tmp = get_mistral_response(
#     argobot_answer='A chicken is black',
#     context='Some chickens are yellow, but the rest are red'
# )
# print(tmp)

In [49]:
import requests

def get_chatbot_answer(question):
    url = 'http://localhost:8000/api/v1/chat/generateAnswer'
    headers = {
        'accept': 'application/json'
    }
    data = {
        'session_id': '1',
        'sender_message': question
    }
    
    files = {
        'file': None
    }

    response = requests.post(url, headers=headers, data=data, files=files)
    
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        return data['response']
    else:
        print(f"Failed to retrieve data: {response.status_code}")

In [50]:
import pandas as pd

dataset = pd.read_excel('kb_data.xlsx')
dataset.head()

,document_id,title,status,created_at,content,id,language,source,source_type,source_size,question
0,19104bca-7f9a-43ea-800f-e23c39d85558,6 cách lựa chọn hạt lúa giống phù hợp,active,1.721800e+09,### 4. Kiểm Tra Chất Lượng Nội Tại của Hạt Lúa...,16991499-eb23-418b-895f-9626efb0ded7,vietnamese,https://hatgiongtruongphuc.vn/blogs/news/6-cac...,url,0.004,Làm sao để xác định độ ẩm của hạt lúa giống?
1,19104bca-7f9a-43ea-800f-e23c39d85558,6 cách lựa chọn hạt lúa giống phù hợp,active,1.721800e+09,### 4. Kiểm Tra Chất Lượng Nội Tại của Hạt Lúa...,16991499-eb23-418b-895f-9626efb0ded7,vietnamese,https://hatgiongtruongphuc.vn/blogs/news/6-cac...,url,0.004,Hàm lượng dinh dưỡng trong hạt lúa giống có ản...
2,19104bca-7f9a-43ea-800f-e23c39d85558,6 cách lựa chọn hạt lúa giống phù hợp,active,1.721800e+09,### 4. Kiểm Tra Chất Lượng Nội Tại của Hạt Lúa...,16991499-eb23-418b-895f-9626efb0ded7,vietnamese,https://hatgiongtruongphuc.vn/blogs/news/6-cac...,url,0.004,Có thể kiểm tra khả năng chống sâu bệnh của lú...
3,19104bca-7f9a-43ea-800f-e23c39d85558,6 cách lựa chọn hạt lúa giống phù hợp,active,1.721800e+09,### 6. Lưu Ý Đến Chu kỳ Sinh Trưởng và Thời Gi...,23175f9f-d38e-4056-a7ae-6037ba96b7ac,vietnamese,https://hatgiongtruongphuc.vn/blogs/news/6-cac...,url,0.004,Chu kỳ sinh trưởng ảnh hưởng đến năng suất cây...
4,19104bca-7f9a-43ea-800f-e23c39d85558,6 cách lựa chọn hạt lúa giống phù hợp,active,1.721800e+09,### 6. Lưu Ý Đến Chu kỳ Sinh Trưởng và Thời Gi...,23175f9f-d38e-4056-a7ae-6037ba96b7ac,vietnamese,https://hatgiongtruongphuc.vn/blogs/news/6-cac...,url,0.004,Cách lựa chọn hạt giống phù hợp với thời gian ...


In [56]:
#evaluate
total_score = 0
for i in range(len(dataset)):
    question = dataset.iloc[i]['question']
    context = dataset.iloc[i]['content']
    
    answer = get_chatbot_answer(question)
    score = get_mistral_response(answer, question, context)
    total_score += score
    print(i, score, '\n', question, '\n', answer)
print(total_score, '/', len(dataset))

0 0 
 Làm sao để xác định độ ẩm của hạt lúa giống? 
 Để xác định độ ẩm của hạt lúa giống, bạn có thể sử dụng hai phương pháp:

1. **Phương pháp sấy khô:**

   * Lấy một lượng hạt lúa giống nhất định (khoảng 100 gram). 
   * Sấy khô hạt lúa trong lò sấy hoặc phơi nắng cho đến khi hạt khô hoàn toàn.
   * Cân trọng lượng hạt lúa sau khi sấy khô.
   * Tính toán độ ẩm bằng công thức: Độ ẩm (%) = (Trọng lượng hạt lúa tươi - Trọng lượng hạt lúa khô) / Trọng lượng hạt lúa tươi x 100%.

2. **Sử dụng thiết bị đo độ ẩm:**

   * Có nhiều loại thiết bị đo độ ẩm chuyên dụng cho hạt lúa, bạn có thể mua tại các cửa hàng nông nghiệp hoặc trực tuyến.
   * Sử dụng thiết bị đo độ ẩm theo hướng dẫn của nhà sản xuất để xác định độ ẩm của hạt lúa giống.

Lưu ý: Độ ẩm của hạt lúa giống lý tưởng để bảo quản và gieo trồng thường từ 12% đến 14%. Độ ẩm quá cao sẽ dễ làm hạt lúa bị mốc, hư hỏng, còn độ ẩm quá thấp sẽ khó nảy mầm.
1 1 
 Hàm lượng dinh dưỡng trong hạt lúa giống có ảnh hưởng gì đến năng suất? 
 Hàm l